### 1. Classic CNN Architectures
These models are early milestones in deep learning, laying the groundwork for many subsequent innovations.

- **AlexNet**: A pioneering deep convolutional neural network, known for using ReLU activation and dropout to prevent overfitting.
- **VGG**: Improves performance with smaller convolutional filters and deeper network structures, simple architecture but large in parameters.

### 2. Residual Networks
Introduced residual connections to build deeper networks, addressing the challenges in training deep architectures.

- **ResNet**: Solves deep network degradation issues with skip connections.
- **ResNeXt**: Based on ResNet, introduces grouped convolutions, enhancing network scalability.
- **Wide ResNet**: A ResNet variant that improves performance by increasing the network's width.

### 3. Inception Series
Effectively controls computational resources using multi-scale convolutional kernels.

- **GoogLeNet (Inception V1)**: Famous for its “Inception” module, using multi-scale convolutions.
- **Inception V3**: An improved version of the Inception model, incorporating batch normalization and label smoothing.

### 4. Mobile and Lightweight Architectures
Designed for mobile and embedded devices, emphasizing efficiency and compactness.

- **MobileNet V2**: Introduces inverted residuals and linear bottlenecks.
- **MobileNet V3**: Combines NAS and NetAdapt techniques for higher efficiency.
- **ShuffleNet V2**: Employs channel shuffle operations, lightweight.
- **SqueezeNet**: Extremely small model size, uses Fire modules.

### 5. Efficient and Adaptive Networks
Use advanced techniques and design philosophies for improved efficiency and performance.

- **DenseNet**: Connects each layer to every other layer, enhancing feature propagation and reuse.
- **EfficientNet**: Scales depth, width, and resolution simultaneously.
- **EfficientNetV2**: An improved version of EfficientNet, optimized for faster training speed.
- **RegNet**: Simplifies network architecture search with regularized design.
- **MNASNet**: Automatically designed through neural architecture search, focusing on mobile efficiency.

### 6. Transformer and Vision Transformer
Applies the Transformer architecture to visual tasks, emphasizing global feature learning.

- **VisionTransformer (ViT)**: Uses image patches as input units for efficient global feature learning.
- **SwinTransformer**: Improves efficiency with a moving window approach.

### 7. Emerging and Hybrid Architectures
Latest network architectures combining different technologies and concepts.

- **ConvNeXt**: Optimizes traditional convolutional network design, inspired by Transformer concepts.
- **MaxVit**: Combines MaxPool and Vision Transformer architectures.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import random

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models, transforms
from torchvision.io import read_image

In [ ]:
class BrainDataset(Dataset):
    def __init__(self, root, phase, augment):
        self.img_dir = root
        self.labels = []
        self.phase = phase
        self.augment = augment
        self.label_map = {'normal': 0, 'glioma_tumor': 1, 'meningioma_tumor': 2, 'pituitary_tumor': 3}

        num_samples={'normal': 400, 'glioma_tumor': 800, 'meningioma_tumor': 800, 'pituitary_tumor': 800}
        data_split = {'train': 0.5, 'val': 0.25, 'test': 0.25}
        for class_dir in os.listdir(self.img_dir):
            class_path = os.path.join(self.img_dir, class_dir)
            if not os.path.isdir(class_path):
                continue

            files = os.listdir(class_path)
            random.shuffle(files)

            num_to_select = num_samples[class_dir]
            subset = files[:num_to_select]
            split_idx = int(len(subset) * data_split[phase])
            if phase == 'train':
                selected_files = subset[:split_idx]
            elif phase == 'val':
                selected_files = subset[split_idx:2*split_idx]
            elif phase == 'test':
                selected_files = subset[2*split_idx:]

            for file_name in selected_files:
                self.labels.append((os.path.join(class_dir, file_name), class_dir))

        if phase == 'train' or phase == 'test':
            self.transforms = transforms.Compose([
                transforms.RandomRotation(15),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
                transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1)]),
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))]),
            ])
        else:
            self.transforms = transforms.Compose([])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels[idx][0])
        image = read_image(img_path).to(torch.float)
        if self.augment:
            image = self.transforms(image)
        label = self.label_map[self.labels[idx][1]]
        label = torch.tensor(label)
        return image, label

def test_dataloader(dataloader):
    for images, labels in dataloader:
        print("Batch size:", len(images))
        print("Image size:", images[0].shape)
        print("Labels:", labels)
        break

root = "./project/Data"
batch_size = 16
train_data = BrainDataset(root=root, phase='train')
val_data = BrainDataset(root=root, phase='val')
test_data = BrainDataset(root=root, phase='test')
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

from torch.backends import cpu
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += int(torch.sum(preds == labels.data))

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)

    return epoch_loss, epoch_acc

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += int(torch.sum(preds == labels.data))

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)

    return epoch_loss, epoch_acc
import copy

def train_loop(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, early_stop_patience):
    train_losses, train_accs, val_losses, val_accs = [], [], [], []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    best_train_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        val_loss, val_acc = validate_model(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # Check if the validation loss improved
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Check if training should be stopped
        if epochs_no_improve >= early_stop_patience:
            print("Validation loss has not improved for {:d} epochs. Stopping training...".format(early_stop_patience))
            break

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model, train_losses, train_accs, val_losses, val_accs

import torch
import numpy as np

def test_model(model, test_loader, num_tests=5):
    model.eval()
    running_corrects = 0

    all_preds = {}

    for _ in range(num_tests):
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size(0)):
                if (i, j) not in all_preds:
                    all_preds[(i, j)] = []
                all_preds[(i, j)].append(preds[j].item())

    for (i, j), preds in all_preds.items():
        label = test_loader.dataset[i][1]
        avg_pred = np.round(np.mean(preds)).astype(int)
        running_corrects += int(avg_pred == label)

    epoch_acc = running_corrects / len(test_loader.dataset)
    return epoch_acc


In [ ]:
model = models.efficientnet_v2_m(weights='DEFAULT', num_classes=1000)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 4)
model = model.to(device)

dataloaders = {'train': train_loader, 'val': val_loader}
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
early_stop_patience = 10
model, train_losses, train_accs, val_losses, val_accs = train_loop(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, device=device, early_stop_patience=early_stop_patience)
actual_epochs = len(train_losses) + 1

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, actual_epochs), train_losses, label='Train Loss')
plt.plot(range(1, actual_epochs), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Val Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, actual_epochs), train_accs, label='Train Acc')
plt.plot(range(1, actual_epochs), val_accs, label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train and Val Accuracy')
plt.legend()

plt.tight_layout()
plt.show()